# Packages and Data Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Stats and other tools
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report,confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import scipy.stats as stats
from scipy.stats import uniform
from scipy.stats import randint as sp_randint
import time
import logging
from sklearn.pipeline import Pipeline
logging.getLogger().setLevel(logging.INFO)

#Models we will test and try
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
import tensorflow as tf
import tensorflow.contrib.learn as learn

#Tensorflow abstractions (for when if we decide for a proper DNN implementation)
from tensorflow import estimator
#from keras import layers
#from tensorflow.contrib.layers import fully_connected

#from keras import models
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import regularizers
#from hyperas.distributions import choice, uniform, conditional
#from hyperas import optim
#from hyperopt import Trials, STATUS_OK, tpe


cv_k_global = 11 #the amount of f_folds to be used in all CV




train_full= pd.read_csv('train.csv')
train_full.set_index('PassengerId',inplace=True)

quantitative = [f for f in train_full.columns if train_full.dtypes[f] != 'object']
quantitative.remove('Survived')#Survived is target label
#quantitative.remove('PassengerId')#PassengerId will be turned into index
qualitative = [f for f in train_full.columns if train_full.dtypes[f] == 'object']

Using TensorFlow backend.


In [21]:
dataset='deck_new'

test_1 = pd.read_csv('data_test_1.csv',index_col='PassengerId').drop(['Missing_Embark','Missing_Deck'],axis=1)
test_2 = pd.read_csv('data_test_2.csv',index_col='PassengerId').drop(['Missing_Embark','Missing_Deck'],axis=1)
test_3 = pd.read_csv('data_test_2.csv',index_col='PassengerId').drop(['Missing_Embark','Missing_Deck'],axis=1)

if(dataset=='old'):
    data_1 = pd.read_csv('data_train_1.csv',index_col='PassengerId').drop(['Missing_Embark','Missing_Deck'],axis=1)
    data_2 = pd.read_csv('data_train_2.csv',index_col='PassengerId').drop(['Missing_Embark','Missing_Deck'],axis=1)
    data_3 = pd.read_csv('data_train_3.csv',index_col='PassengerId').drop(['Missing_Embark','Missing_Deck'],axis=1)
    
if(dataset=='new'):
    data_1 = pd.read_csv('data_train_1_new.csv',index_col='PassengerId').drop(['Missing_Embark','Missing_Deck'],axis=1)
    data_2 = pd.read_csv('data_train_2_new.csv',index_col='PassengerId').drop(['Missing_Embark','Missing_Deck'],axis=1)
    data_3 = pd.read_csv('data_train_3_new.csv',index_col='PassengerId').drop(['Missing_Embark','Missing_Deck'],axis=1)
    
if(dataset=='deck'):
    data_1 = pd.read_csv('data_train_deck_pred_1.csv',index_col='PassengerId').drop(['Missing_Embark','T_deck'],axis=1)
    data_2 = pd.read_csv('data_train_deck_pred_2.csv',index_col='PassengerId').drop(['Missing_Embark','T_deck'],axis=1)
    data_3 = pd.read_csv('data_train_deck_pred_3.csv',index_col='PassengerId').drop(['Missing_Embark','T_deck'],axis=1)
    
if(dataset=='deck_new'):
    data_1 = pd.read_csv('data_train_deck_new_pred_1.csv',index_col='PassengerId').drop(['Missing_Embark','T_deck'],axis=1)
    data_2 = pd.read_csv('data_train_deck_new_pred_2.csv',index_col='PassengerId').drop(['Missing_Embark','T_deck'],axis=1)
    data_3 = pd.read_csv('data_train_deck_new_pred_3.csv',index_col='PassengerId').drop(['Missing_Embark','T_deck'],axis=1)

# Cross-validation and hyper-parameter searches

## Logistic Regression

### Data Type 1

In [3]:
X=data_1.drop('Survived',axis=1).as_matrix()
Y=data_1['Survived'].as_matrix()
logmodel = LogisticRegression(n_jobs=-1)

param_grid = [{
    'C': [10**x for x in range(-3,4)],
    'penalty':['l1'],
    'solver' :['liblinear','saga']},{
    'C': [10**x for x in range(-3,4)],
    'penalty':['l2'],
    'solver':['newton-cg','lbfgs','sag']}]
gscv=GridSearchCV(logmodel,param_grid,scoring='accuracy',cv=cv_k_global,n_jobs=-1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_LR_new_data1.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_LR_data1.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_LR_deck_data1.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_LR_deck_new_data1.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

,mean_fit_time,mean_test_score,mean_train_score,param_C,param_penalty,param_solver,rank_test_score,std_test_score,error
34,0.106099,0.810325,0.814815,1000,l2,sag,1,0.032285,0.016142
31,0.108698,0.810325,0.814815,100,l2,sag,1,0.032285,0.016142
8,0.034762,0.810325,0.814591,10,l1,liblinear,1,0.032777,0.016389
9,0.105913,0.810325,0.814703,10,l1,saga,1,0.032777,0.016389
25,0.108515,0.809203,0.814030,1,l2,sag,5,0.035258,0.017629
26,0.013731,0.809203,0.814815,10,l2,newton-cg,5,0.032815,0.016408
33,0.018787,0.809203,0.814927,1000,l2,lbfgs,5,0.031455,0.015727
27,0.019865,0.809203,0.814703,10,l2,lbfgs,5,0.032815,0.016408
28,0.109425,0.809203,0.814703,10,l2,sag,5,0.032815,0.016408
29,0.015023,0.809203,0.814927,100,l2,newton-cg,5,0.031455,0.015727


### Data Type 2

In [4]:
X=data_2.drop('Survived',axis=1).as_matrix()
Y=data_2['Survived'].as_matrix()
logmodel = LogisticRegression(n_jobs=-1)

param_grid = [{
    'C': [10**x for x in range(-3,4)],
    'penalty':['l1'],
    'solver' :['liblinear','saga']},{
    'C': [10**x for x in range(-3,4)],
    'penalty':['l2'],
    'solver':['newton-cg','lbfgs','sag']}]
gscv=GridSearchCV(logmodel,param_grid,scoring='accuracy',cv=cv_k_global,n_jobs=-1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_LR_new_data2.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_LR_data2.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_LR_deck_data2.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_LR_deck_new_data2.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

,mean_fit_time,mean_test_score,mean_train_score,param_C,param_penalty,param_solver,rank_test_score,std_test_score,error
12,0.119197,0.803591,0.817059,1000,l1,liblinear,1,0.033827,0.016914
32,0.023804,0.803591,0.816834,1000,l2,newton-cg,1,0.033827,0.016914
22,0.114624,0.803591,0.812796,0.1,l2,sag,1,0.044272,0.022136
21,0.008776,0.803591,0.812796,0.1,l2,lbfgs,1,0.044272,0.022136
20,0.011631,0.803591,0.812796,0.1,l2,newton-cg,1,0.044272,0.022136
6,0.043766,0.803591,0.815713,1,l1,liblinear,1,0.040661,0.020330
33,0.023209,0.803591,0.817957,1000,l2,lbfgs,1,0.032998,0.016499
25,0.110748,0.802469,0.816386,1,l2,sag,8,0.040229,0.020114
29,0.021606,0.802469,0.816273,100,l2,newton-cg,8,0.034854,0.017427
23,0.015343,0.802469,0.815938,1,l2,newton-cg,8,0.040229,0.020114


### Data Type 3

In [5]:
X=data_3.drop('Survived',axis=1).as_matrix()
Y=data_3['Survived'].as_matrix()
logmodel = LogisticRegression(n_jobs=-1)

param_grid = [{
    'C': [10**x for x in range(-3,4)],
    'penalty':['l1'],
    'solver' :['liblinear','saga']},{
    'C': [10**x for x in range(-3,4)],
    'penalty':['l2'],
    'solver':['newton-cg','lbfgs','sag']}]
gscv=GridSearchCV(logmodel,param_grid,scoring='accuracy',cv=cv_k_global,n_jobs=-1,verbose=1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_LR_new_data3.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_LR_data3.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_LR_deck_data3.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_LR_deck_new_data3.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 35 candidates, totalling 385 fits


[Parallel(n_jobs=-1)]: Done 385 out of 385 | elapsed:    4.3s finished


,mean_fit_time,mean_test_score,mean_train_score,param_C,param_penalty,param_solver,rank_test_score,std_test_score,error
24,0.018763,0.823793,0.833558,1,l2,lbfgs,1,0.030144,0.015072
23,0.018244,0.823793,0.833558,1,l2,newton-cg,1,0.030144,0.015072
7,0.110437,0.823793,0.832099,1,l1,saga,1,0.034741,0.017370
25,0.110222,0.822671,0.833446,1,l2,sag,4,0.030683,0.015341
8,0.094469,0.822671,0.834230,10,l1,liblinear,4,0.030854,0.015427
6,0.051218,0.822671,0.830977,1,l1,liblinear,4,0.033183,0.016591
27,0.024214,0.821549,0.832659,10,l2,lbfgs,7,0.036082,0.018041
13,0.105911,0.821549,0.835016,1000,l1,saga,7,0.038015,0.019008
34,0.111937,0.821549,0.835353,1000,l2,sag,7,0.038015,0.019008
9,0.112391,0.820426,0.833557,10,l1,saga,10,0.034580,0.017290


## LR With polinomial features

### Data Type 1

In [7]:
X=data_1.drop('Survived',axis=1).as_matrix()
Y=data_1['Survived'].as_matrix()

estimators = [('AddPoly',PolynomialFeatures()),('LR',LogisticRegression())]
quartic_log_reg = Pipeline(estimators)

param_grid = [{
    'AddPoly__degree':[2,3],
    'LR__C': [10**x for x in range(-3,4)],
    'LR__penalty':['l1'],
    'LR__solver' :['liblinear','saga']
    },{
    'AddPoly__degree':[2,3],
    'LR__C': [10**x for x in range(-3,4)],
    'LR__penalty':['l2'],
    'LR__solver' :['newton-cg','lbfgs','sag']
    }]

gscv=GridSearchCV(
    quartic_log_reg,
    param_grid,scoring='accuracy',
    cv=cv_k_global,
    verbose=1,
    n_jobs=-1)

gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_LR_X_new_data1.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_LR_X_data1.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_LR_X_deck_data1.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_LR_X_deck_new_data1.csv')


results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 70 candidates, totalling 770 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 770 out of 770 | elapsed:  2.9min finished


,mean_fit_time,mean_test_score,mean_train_score,param_AddPoly__degree,param_LR__C,param_LR__penalty,param_LR__solver,rank_test_score,std_test_score,error
53,0.503125,0.826038,0.840068,3,0.01,l2,lbfgs,1,0.030458,0.015229
52,1.133583,0.826038,0.840068,3,0.01,l2,newton-cg,1,0.030458,0.015229
60,1.564806,0.819304,0.831538,3,1,l2,sag,3,0.037760,0.018880
23,2.473263,0.819304,0.831763,3,10,l1,saga,3,0.037019,0.018509
34,0.091921,0.818182,0.834904,2,0.1,l2,newton-cg,5,0.039730,0.019865
66,1.630943,0.818182,0.831651,3,100,l2,sag,5,0.036893,0.018447
57,1.493256,0.818182,0.831987,3,0.1,l2,sag,5,0.038421,0.019211
55,2.597667,0.818182,0.856790,3,0.1,l2,newton-cg,5,0.036040,0.018020
35,0.045510,0.818182,0.834904,2,0.1,l2,lbfgs,5,0.039730,0.019865
27,2.100486,0.818182,0.831763,3,1000,l1,saga,5,0.036893,0.018447


### Data Type 2

In [8]:
X=data_2.drop('Survived',axis=1).as_matrix()
Y=data_2['Survived'].as_matrix()

estimators = [('AddPoly',PolynomialFeatures()),('LR',LogisticRegression())]
quartic_log_reg = Pipeline(estimators)

param_grid = [{
    'AddPoly__degree':[2,3],
    'LR__C': [10**x for x in range(-3,4)],
    'LR__penalty':['l1'],
    'LR__solver' :['liblinear','saga']
    },{
    'AddPoly__degree':[2,3],
    'LR__C': [10**x for x in range(-3,4)],
    'LR__penalty':['l2'],
    'LR__solver' :['newton-cg','lbfgs','sag']
    }]

gscv=GridSearchCV(
    quartic_log_reg,
    param_grid,scoring='accuracy',
    cv=cv_k_global,
    verbose=1,
    n_jobs=-1)

gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_LR_X_new_data2.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_LR_X_data2.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_LR_X_deck_data2.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_LR_X_deck_new_data2.csv')


results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 70 candidates, totalling 770 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 631 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 739 out of 770 | elapsed:  8.2min remaining:   20.6s
[Parallel(n_jobs=-1)]: Done 770 out of 770 | elapsed: 11.2min finished


,mean_fit_time,mean_test_score,mean_train_score,param_AddPoly__degree,param_LR__C,param_LR__penalty,param_LR__solver,rank_test_score,std_test_score,error
59,2.801014,0.830527,0.903367,3,1,l2,lbfgs,1,0.031736,0.015868
56,2.509736,0.829405,0.893491,3,0.1,l2,lbfgs,2,0.033245,0.016622
20,0.487451,0.829405,0.893716,3,1,l1,liblinear,2,0.031970,0.015985
6,0.137621,0.827160,0.862178,2,1,l1,liblinear,4,0.043862,0.021931
55,9.799342,0.827160,0.892930,3,0.1,l2,newton-cg,4,0.036496,0.018248
34,0.172818,0.826038,0.851966,2,0.1,l2,newton-cg,6,0.046028,0.023014
35,0.116107,0.826038,0.852190,2,0.1,l2,lbfgs,6,0.046028,0.023014
36,0.354718,0.826038,0.849496,2,0.1,l2,sag,6,0.047800,0.023900
7,0.746128,0.824916,0.846242,2,1,l1,saga,9,0.042960,0.021480
62,3.011275,0.823793,0.905387,3,10,l2,lbfgs,10,0.040169,0.020084


### Data Type 3

In [9]:
X=data_3.drop('Survived',axis=1).as_matrix()
Y=data_3['Survived'].as_matrix()

estimators = [('AddPoly',PolynomialFeatures()),('LR',LogisticRegression())]
quartic_log_reg = Pipeline(estimators)

param_grid = [{
    'AddPoly__degree':[2,3],
    'LR__C': [10**x for x in range(-3,4)],
    'LR__penalty':['l1'],
    'LR__solver' :['liblinear','saga']
    },{
    'AddPoly__degree':[2,3],
    'LR__C': [10**x for x in range(-3,4)],
    'LR__penalty':['l2'],
    'LR__solver' :['newton-cg','lbfgs','sag']
    }]

gscv=GridSearchCV(
    quartic_log_reg,
    param_grid,scoring='accuracy',
    cv=cv_k_global,
    verbose=1,
    n_jobs=-1)

gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_LR_X_new_data3.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_LR_X_data3.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_LR_X_deck_data3.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_LR_X_deck_new_data3.csv')


results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 70 candidates, totalling 770 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 421 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 713 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done 770 out of 770 | elapsed: 20.5min finished


,mean_fit_time,mean_test_score,mean_train_score,param_AddPoly__degree,param_LR__C,param_LR__penalty,param_LR__solver,rank_test_score,std_test_score,error
53,4.016245,0.837262,0.871831,3,0.01,l2,lbfgs,1,0.038231,0.019115
52,10.486533,0.832772,0.872056,3,0.01,l2,newton-cg,2,0.037247,0.018623
7,0.890295,0.829405,0.853536,2,1,l1,saga,3,0.040903,0.020452
6,0.206476,0.829405,0.868237,2,1,l1,liblinear,3,0.037025,0.018512
20,0.412718,0.828283,0.903144,3,1,l1,liblinear,5,0.031688,0.015844
56,3.911456,0.828283,0.905276,3,0.1,l2,lbfgs,5,0.037262,0.018631
34,0.331800,0.827160,0.859596,2,0.1,l2,newton-cg,7,0.044840,0.022420
35,0.197006,0.827160,0.859596,2,0.1,l2,lbfgs,7,0.044840,0.022420
49,7.067519,0.826038,0.847139,3,0.001,l2,newton-cg,9,0.020921,0.010460
55,18.400896,0.826038,0.903817,3,0.1,l2,newton-cg,9,0.038721,0.019361


## AdaBC

### Data Type 1

In [10]:
X=data_1.drop('Survived',axis=1).as_matrix()
Y=data_1['Survived'].as_matrix()

classif_ABC = AdaBoostClassifier()

param_grid ={'n_estimators':[10*x for x in range(1,21)],
             'learning_rate':[10**x for x in range(-4,5)],
             'algorithm':['SAMME','SAMME.R']}
gscv=GridSearchCV(classif_ABC,param_grid,scoring='accuracy',cv=cv_k_global,n_jobs=-1,verbose=1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_ABC_new_data1.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_ABC_data1.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_ABC_deck_data1.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_ABC_deck_new_data1.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 360 candidates, totalling 3960 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 758 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 1013 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done 1363 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done 2260 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-1)]: Done 3360 tasks      | elapsed:   51.1s
[Parallel(n_jobs=-1)]: Done 3960 out of 3960 | elapsed:  1.0min finished


,mean_fit_time,mean_test_score,mean_train_score,param_algorithm,param_learning_rate,param_n_estimators,rank_test_score,std_test_score,error
272,0.271806,0.826038,0.853311,SAMME.R,1,130,1,0.034069,0.017035
271,0.258839,0.823793,0.851404,SAMME.R,1,120,2,0.037045,0.018523
268,0.185324,0.821549,0.846914,SAMME.R,1,90,3,0.032363,0.016181
269,0.232334,0.821549,0.847475,SAMME.R,1,100,3,0.033623,0.016811
279,0.497875,0.820426,0.858923,SAMME.R,1,200,5,0.032664,0.016332
266,0.154660,0.820426,0.844445,SAMME.R,1,70,5,0.022093,0.011046
276,0.349235,0.820426,0.858362,SAMME.R,1,170,5,0.030991,0.015495
267,0.171687,0.820426,0.844108,SAMME.R,1,80,5,0.030063,0.015031
274,0.329068,0.820426,0.855669,SAMME.R,1,150,5,0.030244,0.015122
278,0.407861,0.819304,0.859372,SAMME.R,1,190,10,0.033024,0.016512


### Data Type 2

In [11]:
X=data_2.drop('Survived',axis=1).as_matrix()
Y=data_2['Survived'].as_matrix()

classif_ABC = AdaBoostClassifier()

param_grid ={'n_estimators':[10*x for x in range(1,21)],
             'learning_rate':[10**x for x in range(-4,5)],
             'algorithm':['SAMME','SAMME.R']}
gscv=GridSearchCV(classif_ABC,param_grid,scoring='accuracy',cv=cv_k_global,n_jobs=-1,verbose=1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_ABC_new_data2.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_ABC_data2.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_ABC_deck_data2.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_ABC_deck_new_data2.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 360 candidates, totalling 3960 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 656 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 906 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 1256 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 1706 tasks      | elapsed:   25.6s
[Parallel(n_jobs=-1)]: Done 2256 tasks      | elapsed:   34.4s
[Parallel(n_jobs=-1)]: Done 2906 tasks      | elapsed:   45.9s
[Parallel(n_jobs=-1)]: Done 3656 tasks      | elapsed:   58.4s
[Parallel(n_jobs=-1)]: Done 3960 out of 3960 | elapsed:  1.1min finished


,mean_fit_time,mean_test_score,mean_train_score,param_algorithm,param_learning_rate,param_n_estimators,rank_test_score,std_test_score,error
279,0.460554,0.820426,0.868463,SAMME.R,1,200,1,0.034531,0.017265
270,0.245667,0.819304,0.854096,SAMME.R,1,110,2,0.033832,0.016916
261,0.052341,0.819304,0.830753,SAMME.R,1,20,2,0.028803,0.014401
278,0.414342,0.818182,0.866105,SAMME.R,1,190,4,0.035608,0.017804
277,0.418132,0.818182,0.865993,SAMME.R,1,180,4,0.031915,0.015957
274,0.340986,0.818182,0.859483,SAMME.R,1,150,4,0.035403,0.017702
272,0.290233,0.818182,0.856790,SAMME.R,1,130,4,0.033454,0.016727
263,0.088787,0.818182,0.841751,SAMME.R,1,40,4,0.029978,0.014989
264,0.113059,0.818182,0.841975,SAMME.R,1,50,4,0.025657,0.012829
275,0.358777,0.817059,0.862401,SAMME.R,1,160,10,0.032499,0.016249


### Data Type 3

In [12]:
X=data_3.drop('Survived',axis=1).as_matrix()
Y=data_3['Survived'].as_matrix()

classif_ABC = AdaBoostClassifier()

param_grid ={'n_estimators':[10*x for x in range(1,21)],
             'learning_rate':[10**x for x in range(-4,5)],
             'algorithm':['SAMME','SAMME.R']}
gscv=GridSearchCV(classif_ABC,param_grid,scoring='accuracy',cv=cv_k_global,n_jobs=-1,verbose=1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_ABC_new_data3.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_ABC_data3.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_ABC_deck_data3.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_ABC_deck_new_data3.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 360 candidates, totalling 3960 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 764 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done 1014 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-1)]: Done 1364 tasks      | elapsed:   20.8s
[Parallel(n_jobs=-1)]: Done 2236 tasks      | elapsed:   34.8s
[Parallel(n_jobs=-1)]: Done 3336 tasks      | elapsed:   53.6s
[Parallel(n_jobs=-1)]: Done 3960 out of 3960 | elapsed:  1.1min finished


,mean_fit_time,mean_test_score,mean_train_score,param_algorithm,param_learning_rate,param_n_estimators,rank_test_score,std_test_score,error
83,0.083202,0.827160,0.834232,SAMME,1,40,1,0.024406,0.012203
84,0.103267,0.827160,0.834680,SAMME,1,50,1,0.022019,0.011009
82,0.064242,0.827160,0.833671,SAMME,1,30,1,0.020848,0.010424
81,0.043672,0.826038,0.830640,SAMME,1,20,4,0.026051,0.013025
85,0.131998,0.824916,0.834456,SAMME,1,60,5,0.023135,0.011567
86,0.151525,0.822671,0.835130,SAMME,1,70,6,0.027824,0.013912
260,0.021654,0.821549,0.834232,SAMME.R,1,10,7,0.018532,0.009266
273,0.323212,0.821549,0.868126,SAMME.R,1,140,7,0.036928,0.018464
269,0.218426,0.820426,0.860157,SAMME.R,1,100,9,0.033305,0.016653
278,0.464392,0.820426,0.874522,SAMME.R,1,190,9,0.038979,0.019489


## GBC

### Data Type 1

In [13]:
X=data_1.drop('Survived',axis=1).as_matrix()
Y=data_1['Survived'].as_matrix()

classif_GBC = GradientBoostingClassifier()

param_grid ={'loss' : ['deviance', 'exponential'],
             'learning_rate':[10**x for x in range(-4,5)],
             'criterion':['friedman_mse','mse'],
             'n_estimators':[10*x for x in range(1,21)],
             'subsample':[x*0.1 for x in range(1,11)],
             'max_features':['auto','log2','sqrt',None]}
gscv=GridSearchCV(classif_GBC,param_grid,scoring='accuracy',cv=cv_k_global,n_jobs=-1,verbose=1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_GBC_new_data1.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_GBC_data1.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_GBC_deck_data1.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_GBC_deck_new_data1.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

# scores=cross_val_score(classif_GBC,X=X,y=Y,cv=10)
# print("%0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

Fitting 11 folds for each of 28800 candidates, totalling 316800 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1328 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 2078 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done 3881 tasks      | elapsed:   28.7s
[Parallel(n_jobs=-1)]: Done 6233 tasks      | elapsed:   45.0s
[Parallel(n_jobs=-1)]: Done 7967 tasks      | elapsed:   56.9s
[Parallel(n_jobs=-1)]: Done 9917 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 13028 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 17063 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 19913 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 25292 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 29183 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 35060 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 40316 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 46766 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 52826 tasks      | e

,mean_fit_time,mean_test_score,mean_train_score,param_criterion,param_learning_rate,param_loss,param_max_features,param_n_estimators,param_subsample,rank_test_score,std_test_score,error
20146,0.205977,0.849607,0.905050,mse,0.1,exponential,auto,150,0.7,1,0.041099,0.020549
6348,0.198756,0.847363,0.908082,friedman_mse,0.1,exponential,None,150,0.9,2,0.030879,0.015439
6337,0.189393,0.847363,0.904265,friedman_mse,0.1,exponential,None,140,0.8,2,0.035127,0.017563
19924,0.181995,0.847363,0.908307,mse,0.1,deviance,None,130,0.5,2,0.032819,0.016410
5536,0.191377,0.847363,0.917957,friedman_mse,0.1,deviance,None,140,0.7,2,0.033512,0.016756
20137,0.202992,0.846240,0.903368,mse,0.1,exponential,auto,140,0.8,6,0.035163,0.017581
19396,0.264094,0.846240,0.931762,mse,0.1,deviance,auto,200,0.7,6,0.045567,0.022783
5708,0.149026,0.846240,0.894277,friedman_mse,0.1,exponential,auto,110,0.9,6,0.031475,0.015737
6306,0.167429,0.846240,0.892145,friedman_mse,0.1,exponential,None,110,0.7,6,0.032363,0.016181
20175,0.240794,0.846240,0.906062,mse,0.1,exponential,auto,180,0.6,6,0.033189,0.016594


### Data Type 2

In [14]:
X=data_2.drop('Survived',axis=1).as_matrix()
Y=data_2['Survived'].as_matrix()

classif_GBC = GradientBoostingClassifier()

param_grid ={'loss' : ['deviance', 'exponential'],
             'learning_rate':[10**x for x in range(-4,5)],
             'criterion':['friedman_mse','mse'],
             'n_estimators':[10*x for x in range(1,21)],
             'subsample':[x*0.1 for x in range(1,11)],
             'max_features':['auto','log2','sqrt',None]}
gscv=GridSearchCV(classif_GBC,param_grid,scoring='accuracy',cv=cv_k_global,n_jobs=-1,verbose=1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_GBC_new_data2.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_GBC_data2.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_GBC_deck_data2.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_GBC_deck_new_data2.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

# scores=cross_val_score(classif_GBC,X=X,y=Y,cv=10)
# print("%0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

Fitting 11 folds for each of 28800 candidates, totalling 316800 fits


[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:    0.2s
[Parallel(n_jobs=14)]: Done 1503 tasks      | elapsed:   11.4s
[Parallel(n_jobs=14)]: Done 2003 tasks      | elapsed:   19.4s
[Parallel(n_jobs=14)]: Done 3925 tasks      | elapsed:   33.1s
[Parallel(n_jobs=14)]: Done 5932 tasks      | elapsed:   46.9s
[Parallel(n_jobs=14)]: Done 8799 tasks      | elapsed:  1.3min
[Parallel(n_jobs=14)]: Done 12159 tasks      | elapsed:  1.8min
[Parallel(n_jobs=14)]: Done 15514 tasks      | elapsed:  2.2min
[Parallel(n_jobs=14)]: Done 18331 tasks      | elapsed:  2.7min
[Parallel(n_jobs=14)]: Done 22849 tasks      | elapsed:  3.3min
[Parallel(n_jobs=14)]: Done 27892 tasks      | elapsed:  4.1min
[Parallel(n_jobs=14)]: Done 33214 tasks      | elapsed:  4.8min
[Parallel(n_jobs=14)]: Done 38836 tasks      | elapsed:  5.7min
[Parallel(n_jobs=14)]: Done 45484 tasks      | elapsed:  6.7min
[Parallel(n_jobs=14)]: Done 52273 tasks      | elapsed:  7.7min
[Parallel(n_jobs=14)]: Done 60172 tasks      | 

,mean_fit_time,mean_test_score,mean_train_score,param_criterion,param_learning_rate,param_loss,param_max_features,param_n_estimators,param_subsample,rank_test_score,std_test_score,error
6316,0.174387,0.845118,0.904939,friedman_mse,0.1,exponential,None,120,0.7,1,0.036035,0.018018
19352,0.193289,0.842873,0.917171,mse,0.1,deviance,auto,160,0.3,2,0.032855,0.016427
5523,0.166768,0.842873,0.914815,friedman_mse,0.1,deviance,None,130,0.4,2,0.037383,0.018692
19343,0.195273,0.841751,0.920875,mse,0.1,deviance,auto,150,0.4,4,0.027696,0.013848
5716,0.176801,0.841751,0.907071,friedman_mse,0.1,exponential,auto,120,0.7,4,0.041555,0.020778
19365,0.239443,0.841751,0.930304,mse,0.1,deviance,auto,170,0.6,4,0.030313,0.015156
5697,0.144768,0.841751,0.901234,friedman_mse,0.1,exponential,auto,100,0.8,4,0.038808,0.019404
19793,0.183786,0.841751,0.905276,mse,0.1,deviance,sqrt,200,0.4,4,0.037046,0.018523
20745,0.216594,0.841751,0.910887,mse,0.1,exponential,None,150,0.6,4,0.033922,0.016961
5976,0.179058,0.840629,0.898878,friedman_mse,0.1,exponential,log2,180,0.7,10,0.035503,0.017751


### Data Type 3

In [15]:
X=data_3.drop('Survived',axis=1).as_matrix()
Y=data_3['Survived'].as_matrix()

classif_GBC = GradientBoostingClassifier()

param_grid ={'loss' : ['deviance', 'exponential'],
             'learning_rate':[10**x for x in range(-4,5)],
             'criterion':['friedman_mse','mse'],
             'n_estimators':[10*x for x in range(1,21)],
             'subsample':[x*0.1 for x in range(1,11)],
             'max_features':['auto','log2','sqrt',None]}
gscv=GridSearchCV(classif_GBC,param_grid,scoring='accuracy',cv=cv_k_global,n_jobs=-1,verbose=1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_GBC_new_data3.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_GBC_data3.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_GBC_deck_data3.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_GBC_deck_new_data3.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

# scores=cross_val_score(classif_GBC,X=X,y=Y,cv=10)
# print("%0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

Fitting 11 folds for each of 28800 candidates, totalling 316800 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1222 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 1728 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 2516 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done 4443 tasks      | elapsed:   38.5s
[Parallel(n_jobs=-1)]: Done 7157 tasks      | elapsed:   56.2s
[Parallel(n_jobs=-1)]: Done 9503 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 11753 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 15113 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 17963 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 21113 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 25403 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 29153 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 33968 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 38318 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 43829 tasks      | e

,mean_fit_time,mean_test_score,mean_train_score,param_criterion,param_learning_rate,param_loss,param_max_features,param_n_estimators,param_subsample,rank_test_score,std_test_score,error
19933,0.197607,0.847363,0.918406,mse,0.1,deviance,None,140,0.4,1,0.031872,0.015936
6334,0.241631,0.847363,0.903929,friedman_mse,0.1,exponential,None,140,0.5,1,0.035930,0.017965
6372,0.256820,0.846240,0.902245,friedman_mse,0.1,exponential,None,180,0.3,3,0.025731,0.012866
19295,0.157904,0.846240,0.910775,mse,0.1,deviance,auto,100,0.6,3,0.031830,0.015915
6294,0.170797,0.846240,0.893266,friedman_mse,0.1,exponential,None,100,0.5,3,0.034801,0.017400
20142,0.211613,0.845118,0.895624,mse,0.1,exponential,auto,150,0.3,6,0.028690,0.014345
19343,0.226401,0.845118,0.921326,mse,0.1,deviance,auto,150,0.4,6,0.038919,0.019459
4913,0.181931,0.845118,0.910776,friedman_mse,0.1,deviance,auto,120,0.4,6,0.036560,0.018280
20724,0.209999,0.845118,0.901908,mse,0.1,exponential,None,130,0.5,6,0.025205,0.012602
4974,0.286728,0.845118,0.934344,friedman_mse,0.1,deviance,auto,180,0.5,6,0.027890,0.013945


## Random Forests

### Data Type 1

In [16]:
X=data_1.drop('Survived',axis=1).as_matrix()
Y=data_1['Survived'].as_matrix()
rfc = RandomForestClassifier(n_jobs=-1)

param_grid ={
    'n_estimators':[10*x for x in range(1,21)],
    'criterion':['gini','entropy'],
    'max_features':['auto','log2','sqrt',None]}
gscv=GridSearchCV(rfc,param_grid,scoring='accuracy',cv=cv_k_global,verbose=1,n_jobs=-1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_RF_new_data1.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_RF_data1.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_RF_deck_data1.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_RF_deck_new_data1.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 160 candidates, totalling 1760 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:   45.7s
[Parallel(n_jobs=-1)]: Done 1760 out of 1760 | elapsed:  1.1min finished


,mean_fit_time,mean_test_score,mean_train_score,param_criterion,param_max_features,param_n_estimators,rank_test_score,std_test_score,error
75,0.577216,0.832772,0.983165,gini,None,160,1,0.049188,0.024594
78,0.678910,0.831650,0.983165,gini,None,190,2,0.050585,0.025292
149,0.392089,0.830527,0.982828,entropy,None,100,3,0.052893,0.026446
74,0.524206,0.830527,0.983165,gini,None,150,3,0.047559,0.023780
63,0.176977,0.830527,0.981931,gini,None,40,3,0.050395,0.025198
79,0.702308,0.829405,0.983165,gini,None,200,6,0.048500,0.024250
150,0.448106,0.828283,0.983053,entropy,None,110,7,0.052346,0.026173
77,0.676076,0.828283,0.983165,gini,None,180,7,0.050820,0.025410
158,0.806874,0.827160,0.983165,entropy,None,190,9,0.049501,0.024751
76,0.596018,0.827160,0.983165,gini,None,170,9,0.054946,0.027473


### Data Type 2

In [17]:
X=data_2.drop('Survived',axis=1).as_matrix()
Y=data_2['Survived'].as_matrix()
rfc = RandomForestClassifier(n_jobs=-1)

param_grid ={
    'n_estimators':[10*x for x in range(1,21)],
    'criterion':['gini','entropy'],
    'max_features':['auto','log2','sqrt',None]}
gscv=GridSearchCV(rfc,param_grid,scoring='accuracy',cv=cv_k_global,verbose=1,n_jobs=-1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_RF_new_data2.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_RF_data2.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_RF_deck_data2.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_RF_deck_new_data2.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 160 candidates, totalling 1760 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:   46.3s
[Parallel(n_jobs=-1)]: Done 1760 out of 1760 | elapsed:  1.2min finished


,mean_fit_time,mean_test_score,mean_train_score,param_criterion,param_max_features,param_n_estimators,rank_test_score,std_test_score,error
140,0.143198,0.828283,0.969921,entropy,None,10,1,0.051180,0.025590
158,0.882472,0.827160,0.988552,entropy,None,190,2,0.045777,0.022888
148,0.415620,0.826038,0.988440,entropy,None,90,3,0.056370,0.028185
78,0.788873,0.824916,0.988552,gini,None,190,4,0.055931,0.027965
156,0.797362,0.823793,0.988552,entropy,None,170,5,0.047585,0.023792
76,0.701878,0.823793,0.988552,gini,None,170,5,0.044913,0.022456
157,0.932326,0.822671,0.988552,entropy,None,180,7,0.047927,0.023964
155,0.793269,0.822671,0.988552,entropy,None,160,7,0.050394,0.025197
152,0.645523,0.822671,0.988440,entropy,None,130,7,0.053254,0.026627
150,0.479896,0.822671,0.988552,entropy,None,110,7,0.050419,0.025210


### Data Type 3

In [18]:
X=data_3.drop('Survived',axis=1).as_matrix()
Y=data_3['Survived'].as_matrix()
rfc = RandomForestClassifier(n_jobs=-1)

param_grid ={
    'n_estimators':[10*x for x in range(1,21)],
    'criterion':['gini','entropy'],
    'max_features':['auto','log2','sqrt',None]}
gscv=GridSearchCV(rfc,param_grid,scoring='accuracy',cv=cv_k_global,verbose=1,n_jobs=-1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_RF_new_data3.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_RF_data3.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_RF_deck_data3.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_RF_deck_new_data3.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 160 candidates, totalling 1760 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:   28.3s
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:   46.6s
[Parallel(n_jobs=-1)]: Done 1760 out of 1760 | elapsed:  1.2min finished


,mean_fit_time,mean_test_score,mean_train_score,param_criterion,param_max_features,param_n_estimators,rank_test_score,std_test_score,error
156,0.875568,0.832772,0.989562,entropy,None,170,1,0.042260,0.021130
63,0.170823,0.829405,0.987542,gini,None,40,2,0.048031,0.024016
157,0.839426,0.828283,0.989562,entropy,None,180,3,0.043580,0.021790
154,0.761990,0.828283,0.989562,entropy,None,150,3,0.045986,0.022993
70,0.419559,0.828283,0.989562,gini,None,110,3,0.042087,0.021043
159,0.877269,0.827160,0.989562,entropy,None,200,6,0.044693,0.022347
75,0.649114,0.827160,0.989562,gini,None,160,6,0.041477,0.020739
149,0.460919,0.827160,0.989450,entropy,None,100,6,0.043364,0.021682
78,0.789299,0.826038,0.989562,gini,None,190,9,0.045448,0.022724
76,0.709360,0.826038,0.989562,gini,None,170,9,0.045142,0.022571


## KNN

### Data Type 1

In [19]:
X=data_1.drop('Survived',axis=1).as_matrix()
Y=data_1['Survived'].as_matrix()
knn = KNeighborsClassifier()

param_grid = [{
    'n_neighbors':[2*x+1 for x in range(1,51)],
    'algorithm':[ 'ball_tree', 'kd_tree','brute'],
    'p':[1,2],
    'weights':['uniform','distance'],
    'leaf_size':[2*x+1 for x in range(1,51)]}]
gscv=GridSearchCV(knn,param_grid,scoring='accuracy',cv=cv_k_global,verbose=1,n_jobs=-1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_KNN_new_data1.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_KNN_data1.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_KNN_deck_data1.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_KNN_deck_new_data1.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 30000 candidates, totalling 330000 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1392 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 3892 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done 7392 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 11892 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done 17392 tasks      | elapsed:   43.8s
[Parallel(n_jobs=-1)]: Done 23892 tasks      | elapsed:   59.4s
[Parallel(n_jobs=-1)]: Done 31392 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 39892 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 49392 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 59892 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 71392 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 83892 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 97392 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 111892 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 120447 tasks    

,mean_fit_time,mean_test_score,mean_train_score,param_algorithm,param_leaf_size,param_n_neighbors,param_p,param_weights,rank_test_score,std_test_score,error
11031,0.000845,0.804714,0.983165,kd_tree,13,17,2,distance,1,0.039853,0.019927
15031,0.000680,0.804714,0.983165,kd_tree,53,17,2,distance,1,0.039853,0.019927
12231,0.000844,0.804714,0.983165,kd_tree,25,17,2,distance,1,0.039853,0.019927
16831,0.000678,0.804714,0.983165,kd_tree,71,17,2,distance,1,0.039853,0.019927
28831,0.001184,0.804714,0.983165,brute,91,17,2,distance,1,0.039853,0.019927
5036,0.000747,0.804714,0.816946,ball_tree,53,21,1,uniform,1,0.040175,0.020087
5031,0.000699,0.804714,0.983165,ball_tree,53,17,2,distance,1,0.039853,0.019927
7636,0.000935,0.804714,0.816946,ball_tree,79,21,1,uniform,1,0.040175,0.020087
7236,0.000879,0.804714,0.816946,ball_tree,75,21,1,uniform,1,0.040175,0.020087
18431,0.001276,0.804714,0.983165,kd_tree,87,17,2,distance,1,0.039853,0.019927


### Data Type 2

In [20]:
X=data_2.drop('Survived',axis=1).as_matrix()
Y=data_2['Survived'].as_matrix()
knn = KNeighborsClassifier()

param_grid = [{
    'n_neighbors':[2*x+1 for x in range(1,51)],
    'algorithm':[ 'ball_tree', 'kd_tree','brute'],
    'p':[1,2],
    'weights':['uniform','distance'],
    'leaf_size':[2*x+1 for x in range(1,51)]}]
gscv=GridSearchCV(knn,param_grid,scoring='accuracy',cv=cv_k_global,verbose=1,n_jobs=-1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_KNN_new_data2.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_KNN_data2.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_KNN_deck_data2.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_KNN_deck_new_data2.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 30000 candidates, totalling 330000 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1664 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 4664 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 8864 tasks      | elapsed:   26.9s
[Parallel(n_jobs=-1)]: Done 14264 tasks      | elapsed:   41.3s
[Parallel(n_jobs=-1)]: Done 20864 tasks      | elapsed:   59.1s
[Parallel(n_jobs=-1)]: Done 28664 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 37664 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 47864 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 59264 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 71864 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 85664 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 100664 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 114014 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 122714 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 132014 tasks  

,mean_fit_time,mean_test_score,mean_train_score,param_algorithm,param_leaf_size,param_n_neighbors,param_p,param_weights,rank_test_score,std_test_score,error
14608,0.000857,0.817059,0.854432,kd_tree,49,7,1,uniform,1,0.044247,0.022124
19608,0.001020,0.817059,0.854432,kd_tree,99,7,1,uniform,1,0.044247,0.022124
4808,0.000872,0.817059,0.854432,ball_tree,51,7,1,uniform,1,0.044247,0.022124
9208,0.000830,0.817059,0.854432,ball_tree,95,7,1,uniform,1,0.044247,0.022124
6408,0.000869,0.817059,0.854432,ball_tree,67,7,1,uniform,1,0.044247,0.022124
16608,0.000796,0.817059,0.854432,kd_tree,69,7,1,uniform,1,0.044247,0.022124
2008,0.001117,0.817059,0.854320,ball_tree,23,7,1,uniform,1,0.044247,0.022124
408,0.001128,0.817059,0.854096,ball_tree,7,7,1,uniform,1,0.043298,0.021649
11408,0.000960,0.817059,0.854208,kd_tree,17,7,1,uniform,1,0.044247,0.022124
18608,0.000829,0.817059,0.854432,kd_tree,89,7,1,uniform,1,0.044247,0.022124


### Data Type 3

In [21]:
X=data_3.drop('Survived',axis=1).as_matrix()
Y=data_3['Survived'].as_matrix()
knn = KNeighborsClassifier()

param_grid = [{
    'n_neighbors':[2*x+1 for x in range(1,51)],
    'algorithm':[ 'ball_tree', 'kd_tree','brute'],
    'p':[1,2],
    'weights':['uniform','distance'],
    'leaf_size':[2*x+1 for x in range(1,51)]}]
gscv=GridSearchCV(knn,param_grid,scoring='accuracy',cv=cv_k_global,verbose=1,n_jobs=-1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

if(dataset=='new'):
    results_df.to_csv('./CV_KNN_new_data3.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_KNN_data3.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_KNN_deck_data3.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_KNN_deck_new_data3.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 30000 candidates, totalling 330000 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1392 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 3892 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 7392 tasks      | elapsed:   24.3s
[Parallel(n_jobs=-1)]: Done 11892 tasks      | elapsed:   37.9s
[Parallel(n_jobs=-1)]: Done 17392 tasks      | elapsed:   54.5s
[Parallel(n_jobs=-1)]: Done 23892 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 31392 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 39892 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 49392 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 59892 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 71392 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 83892 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 97392 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 111452 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 119202 tasks    

,mean_fit_time,mean_test_score,mean_train_score,param_algorithm,param_leaf_size,param_n_neighbors,param_p,param_weights,rank_test_score,std_test_score,error
4,0.001497,0.829405,0.867678,ball_tree,3,5,1,uniform,1,0.045631,0.022815
10004,0.001484,0.829405,0.867902,kd_tree,3,5,1,uniform,1,0.045631,0.022815
26204,0.001789,0.828283,0.868127,brute,65,5,1,uniform,3,0.044951,0.022476
25204,0.003986,0.828283,0.868127,brute,55,5,1,uniform,3,0.044951,0.022476
12004,0.001127,0.828283,0.867790,kd_tree,23,5,1,uniform,3,0.047017,0.023509
4804,0.000952,0.828283,0.868127,ball_tree,51,5,1,uniform,3,0.047017,0.023509
21004,0.002252,0.828283,0.868127,brute,13,5,1,uniform,3,0.044951,0.022476
22004,0.001906,0.828283,0.868127,brute,23,5,1,uniform,3,0.044951,0.022476
1604,0.001104,0.828283,0.867790,ball_tree,19,5,1,uniform,3,0.047017,0.023509
25404,0.001983,0.828283,0.868127,brute,57,5,1,uniform,3,0.044951,0.022476


## SVM

### Data Type 1

In [22]:
X=data_1.drop('Survived',axis=1).as_matrix()
Y=data_1['Survived'].as_matrix()
svm_class = svm.SVC()

param_grid =[
    {'C':np.linspace(0.001,1000,21),
    'kernel':['poly'],
    'degree':[2,3,4,5],
    'decision_function_shape':['ovo','ovr']},
     {'C':np.linspace(0.001,1000,21),
    'kernel':['rbf','linear','sigmoid'],
    'decision_function_shape':['ovo','ovr']}
    ]
gscv=GridSearchCV(svm_class,param_grid,scoring='accuracy',cv=cv_k_global,verbose=1,n_jobs=-1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)
    
if(dataset=='new'):
    results_df.to_csv('./CV_SVM_new_data1.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_SVM_data1.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_SVM_deck_data1.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_SVM_deck_new_data1.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 294 candidates, totalling 3234 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 1588 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done 2192 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 2642 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 13.8min
[Parallel(n_jobs=-1)]: Done 3234 out of 3234 | elapsed: 15.0min finished


,mean_fit_time,mean_test_score,mean_train_score,param_C,param_decision_function_shape,param_degree,param_kernel,rank_test_score,std_test_score,error
264,0.199787,0.828283,0.892706,800,ovo,NaN,rbf,1,0.035111,0.017555
267,0.228117,0.828283,0.892706,800,ovr,NaN,rbf,1,0.035111,0.017555
237,0.168200,0.827160,0.891583,550,ovr,NaN,rbf,3,0.035256,0.017628
258,0.165511,0.827160,0.892257,750,ovo,NaN,rbf,3,0.034532,0.017266
240,0.158137,0.827160,0.891471,600,ovo,NaN,rbf,3,0.035256,0.017628
243,0.141340,0.827160,0.891471,600,ovr,NaN,rbf,3,0.035256,0.017628
261,0.188198,0.827160,0.892257,750,ovr,NaN,rbf,3,0.034532,0.017266
234,0.137647,0.827160,0.891583,550,ovo,NaN,rbf,3,0.035256,0.017628
231,0.131215,0.826038,0.891471,500,ovr,NaN,rbf,9,0.033834,0.016917
246,0.141284,0.826038,0.892481,650,ovo,NaN,rbf,9,0.036588,0.018294


### Data Type 2

In [23]:
X=data_2.drop('Survived',axis=1).as_matrix()
Y=data_2['Survived'].as_matrix()
svm_class = svm.SVC()

param_grid =[
    {'C':np.linspace(0.001,1000,21),
    'kernel':['poly'],
    'degree':[2,3,4,5],
    'decision_function_shape':['ovo','ovr']},
     {'C':np.linspace(0.001,1000,21),
    'kernel':['rbf','linear','sigmoid'],
    'decision_function_shape':['ovo','ovr']}
    ]
gscv=GridSearchCV(svm_class,param_grid,scoring='accuracy',cv=cv_k_global,verbose=1,n_jobs=-1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)
    
if(dataset=='new'):
    results_df.to_csv('./CV_SVM_new_data2.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_SVM_data2.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_SVM_deck_data2.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_SVM_deck_new_data2.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 294 candidates, totalling 3234 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 848 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 1712 tasks      | elapsed:   18.0s
[Parallel(n_jobs=-1)]: Done 2326 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 2776 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 3234 out of 3234 | elapsed: 12.4min finished


,mean_fit_time,mean_test_score,mean_train_score,param_C,param_decision_function_shape,param_degree,param_kernel,rank_test_score,std_test_score,error
192,0.084839,0.833895,0.902582,200.001,ovo,NaN,rbf,1,0.035271,0.017635
195,0.090749,0.833895,0.902582,200.001,ovr,NaN,rbf,1,0.035271,0.017635
204,0.119990,0.833895,0.905837,300.001,ovo,NaN,rbf,1,0.031174,0.015587
207,0.136150,0.833895,0.905837,300.001,ovr,NaN,rbf,1,0.031174,0.015587
35,0.088499,0.831650,0.903480,200.001,ovo,5,poly,5,0.046699,0.023350
31,0.065497,0.831650,0.899663,150.001,ovr,5,poly,5,0.041496,0.020748
39,0.074429,0.831650,0.903480,200.001,ovr,5,poly,5,0.046699,0.023350
198,0.106285,0.831650,0.904153,250.001,ovo,NaN,rbf,5,0.035626,0.017813
201,0.117721,0.831650,0.904153,250.001,ovr,NaN,rbf,5,0.035626,0.017813
210,0.117854,0.831650,0.906062,350.001,ovo,NaN,rbf,5,0.031575,0.015788


### Data Type 3

In [24]:
X=data_3.drop('Survived',axis=1).as_matrix()
Y=data_3['Survived'].as_matrix()
svm_class = svm.SVC()

param_grid =[
    {'C':np.linspace(0.001,1000,21),
    'kernel':['poly'],
    'degree':[2,3,4,5],
    'decision_function_shape':['ovo','ovr']},
     {'C':np.linspace(0.001,1000,21),
    'kernel':['rbf','linear','sigmoid'],
    'decision_function_shape':['ovo','ovr']}
    ]
gscv=GridSearchCV(svm_class,param_grid,scoring='accuracy',cv=cv_k_global,verbose=1,n_jobs=-1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)
    
if(dataset=='new'):
    results_df.to_csv('./CV_SVM_new_data3.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_SVM_data3.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_SVM_deck_data3.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_SVM_deck_new_data3.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 294 candidates, totalling 3234 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 848 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 1967 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done 2417 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 2867 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3234 out of 3234 | elapsed:  8.6min finished


,mean_fit_time,mean_test_score,mean_train_score,param_C,param_decision_function_shape,param_degree,param_kernel,rank_test_score,std_test_score,error
93,0.134373,0.831650,0.912346,550,ovr,3,poly,1,0.024276,0.012138
89,0.130541,0.831650,0.912346,550,ovo,3,poly,1,0.024276,0.012138
57,0.101373,0.830527,0.909989,350.001,ovo,3,poly,3,0.023592,0.011796
61,0.108693,0.830527,0.909989,350.001,ovr,3,poly,3,0.023592,0.011796
65,0.111956,0.830527,0.910663,400.001,ovo,3,poly,3,0.024739,0.012369
69,0.105097,0.830527,0.910663,400.001,ovr,3,poly,3,0.024739,0.012369
81,0.124213,0.830527,0.911785,500,ovo,3,poly,3,0.024739,0.012369
85,0.119336,0.830527,0.911785,500,ovr,3,poly,3,0.024739,0.012369
73,0.120796,0.829405,0.910663,450.001,ovo,3,poly,9,0.025688,0.012844
77,0.107657,0.829405,0.910663,450.001,ovr,3,poly,9,0.025688,0.012844


## DNN in Keras

In [3]:
def my_DNN_classifier(in_layer,optimizer='adam',neurons=64,dropout=0.0,activation='relu',loss='binary_crossentropy',activation_final='sigmoid',shape='one'):
    model=None
    model = Sequential()
    model.add(Dense(neurons, input_dim=in_layer, kernel_initializer='normal',activation=activation))
    model.add(Dropout(dropout))
    if(shape=='none'):
        None
    if(shape=='one'):
        model.add(Dense(neurons, kernel_initializer='normal',activation=activation))
        model.add(Dropout(dropout))
    if(shape=='two'):
        model.add(Dense(neurons, kernel_initializer='normal',activation=activation))
        model.add(Dropout(dropout))
        model.add(Dense(neurons, kernel_initializer='normal',activation=activation))
        model.add(Dropout(dropout))
    model.add(Dense(1, kernel_initializer='normal', activation=activation_final))
    model.compile(loss=loss, metrics=['accuracy'],optimizer=optimizer)
    return model

### Data Type 1

In [4]:
X=data_1.drop('Survived',axis=1).as_matrix()
Y=data_1['Survived'].as_matrix()

classifier = None
classifier = KerasClassifier(build_fn=my_DNN_classifier,verbose=0)

param_grid ={'batch_size': [1024],
                      'epochs': [1000],
                      'in_layer' :[X.shape[1]],
                      'optimizer': ['adam','adagrad','rmsprop','adadelta','nadam'],# ['adam','adagrad']
                      'dropout' : np.linspace(0,1,11),#[0.0,0.1,0.2,0.3,0.4,0.5]
                      'neurons' : [32],#sp_randint(1, 101)
                      'activation' : ['relu'],#['relu','sigmoid','tanh']
                      'activation_final' : ['sigmoid'],#['relu','sigmoid','tanh']
                      'loss' : ['binary_crossentropy'],#['sparse_categorical_crossentropy','binary_crossentropy'],)
                      'shape':['two']
                     }
gscv=GridSearchCV(estimator=classifier,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=5,
                  verbose=1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

results_df['error']=results_df['std_test_score']/2

del classifier

if(dataset=='new'):
    results_df.to_csv('./CV_DNN_new_data1.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_DNN_data1.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_DNN_deck_data1.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_DNN_deck_new_data1.csv')

results_df.sort_values('rank_test_score',axis=0).drop(['param_epochs','param_loss','param_batch_size','param_in_layer','mean_score_time','params','split0_test_score', 'split0_train_score', 'split1_test_score',
       'split1_train_score', 'split2_test_score', 'split2_train_score',
       'split3_test_score', 'split3_train_score', 'split4_test_score',
       'split4_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 5 folds for each of 55 candidates, totalling 275 fits


[Parallel(n_jobs=1)]: Done 275 out of 275 | elapsed: 75.9min finished


,mean_fit_time,mean_test_score,mean_train_score,param_activation,param_activation_final,param_dropout,param_neurons,param_optimizer,param_shape,rank_test_score,std_test_score,error
8,9.632378,0.819304,0.885801,relu,sigmoid,0.1,32,adadelta,two,1,0.028161,0.014080
11,9.034875,0.817059,0.875422,relu,sigmoid,0.2,32,adagrad,two,2,0.036483,0.018242
17,11.966596,0.817059,0.884121,relu,sigmoid,0.3,32,rmsprop,two,2,0.033101,0.016551
33,18.626352,0.817059,0.871214,relu,sigmoid,0.6,32,adadelta,two,2,0.025150,0.012575
30,13.662642,0.815937,0.872337,relu,sigmoid,0.6,32,adam,two,5,0.028000,0.014000
53,20.715109,0.814815,0.879625,relu,sigmoid,1,32,adadelta,two,6,0.030137,0.015068
23,14.847398,0.814815,0.883278,relu,sigmoid,0.4,32,adadelta,two,6,0.024349,0.012174
32,14.665086,0.814815,0.870092,relu,sigmoid,0.6,32,rmsprop,two,6,0.023292,0.011646
28,13.635759,0.813692,0.870369,relu,sigmoid,0.5,32,adadelta,two,9,0.035922,0.017961
38,17.849740,0.813692,0.850170,relu,sigmoid,0.7,32,adadelta,two,9,0.027151,0.013576


### Data Type 2

In [5]:
X=data_2.drop('Survived',axis=1).as_matrix()
Y=data_2['Survived'].as_matrix()

classifier = None
classifier = KerasClassifier(build_fn=my_DNN_classifier,verbose=0)

param_grid ={'batch_size': [1024],
                      'epochs': [1000],
                      'in_layer' :[X.shape[1]],
                      'optimizer': ['adam','adagrad','rmsprop','adadelta','nadam'],# ['adam','adagrad']
                      'dropout' : np.linspace(0,1,11),#[0.0,0.1,0.2,0.3,0.4,0.5]
                      'neurons' : [32],#sp_randint(1, 101)
                      'activation' : ['relu'],#['relu','sigmoid','tanh']
                      'activation_final' : ['sigmoid'],#['relu','sigmoid','tanh']
                      'loss' : ['binary_crossentropy'],#['sparse_categorical_crossentropy','binary_crossentropy'],)
                      'shape':['two']
                     }
gscv=GridSearchCV(estimator=classifier,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=5,
                  verbose=1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

del classifier

results_df['error']=results_df['std_test_score']/2

if(dataset=='new'):
    results_df.to_csv('./CV_DNN_new_data2.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_DNN_data2.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_DNN_deck_data2.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_DNN_deck_new_data2.csv')

results_df.sort_values('rank_test_score',axis=0).drop(['param_epochs','param_loss','param_batch_size','param_in_layer','mean_score_time','params','split0_test_score', 'split0_train_score', 'split1_test_score',
       'split1_train_score', 'split2_test_score', 'split2_train_score',
       'split3_test_score', 'split3_train_score', 'split4_test_score',
       'split4_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 5 folds for each of 55 candidates, totalling 275 fits


[Parallel(n_jobs=1)]: Done 275 out of 275 | elapsed: 73.6min finished


,mean_fit_time,mean_test_score,mean_train_score,param_activation,param_activation_final,param_dropout,param_neurons,param_optimizer,param_shape,rank_test_score,std_test_score,error
23,13.426247,0.838384,0.908531,relu,sigmoid,0.4,32,adadelta,two,1,0.032721,0.016361
27,13.453851,0.833895,0.902076,relu,sigmoid,0.5,32,rmsprop,two,2,0.036081,0.018040
33,16.680633,0.832772,0.897868,relu,sigmoid,0.6,32,adadelta,two,3,0.035129,0.017564
18,11.521998,0.832772,0.906287,relu,sigmoid,0.3,32,adadelta,two,3,0.035629,0.017814
34,16.931480,0.831650,0.906006,relu,sigmoid,0.6,32,nadam,two,5,0.033369,0.016684
22,11.740880,0.831650,0.906285,relu,sigmoid,0.4,32,rmsprop,two,5,0.031743,0.015872
30,15.137443,0.831650,0.898429,relu,sigmoid,0.6,32,adam,two,5,0.040687,0.020344
28,15.273497,0.830527,0.900957,relu,sigmoid,0.5,32,adadelta,two,8,0.026464,0.013232
31,14.486628,0.830527,0.890293,relu,sigmoid,0.6,32,adagrad,two,8,0.033377,0.016688
12,9.196900,0.829405,0.904602,relu,sigmoid,0.2,32,rmsprop,two,10,0.043419,0.021710


### Data Type 3

In [4]:
X=data_3.drop('Survived',axis=1).as_matrix()
Y=data_3['Survived'].as_matrix()

classifier = None
classifier = KerasClassifier(build_fn=my_DNN_classifier,verbose=0)

param_grid ={'batch_size': [1024],
                      'epochs': [1000],
                      'in_layer' :[X.shape[1]],
                      'optimizer': ['adam','adagrad','rmsprop','adadelta','nadam'],# ['adam','adagrad']
                      'dropout' : np.linspace(0,1,11),#[0.0,0.1,0.2,0.3,0.4,0.5]
                      'neurons' : [32],#sp_randint(1, 101)
                      'activation' : ['relu'],#['relu','sigmoid','tanh']
                      'activation_final' : ['sigmoid'],#['relu','sigmoid','tanh']
                      'loss' : ['binary_crossentropy'],#['sparse_categorical_crossentropy','binary_crossentropy'],)
                      'shape':['two']
                     }
gscv=GridSearchCV(estimator=classifier,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=5,
                  verbose=1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)

del classifier

results_df['error']=results_df['std_test_score']/2

if(dataset=='new'):
    results_df.to_csv('./CV_DNN_new_data3.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_DNN_data3.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_DNN_deck_data3.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_DNN_deck_new_data3.csv')

results_df.sort_values('rank_test_score',axis=0).drop(['param_epochs','param_loss','param_batch_size','param_in_layer','mean_score_time','params','split0_test_score', 'split0_train_score', 'split1_test_score',
       'split1_train_score', 'split2_test_score', 'split2_train_score',
       'split3_test_score', 'split3_train_score', 'split4_test_score',
       'split4_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 5 folds for each of 55 candidates, totalling 275 fits


[Parallel(n_jobs=1)]: Done 275 out of 275 | elapsed: 75.5min finished


,mean_fit_time,mean_test_score,mean_train_score,param_activation,param_activation_final,param_dropout,param_neurons,param_optimizer,param_shape,rank_test_score,std_test_score,error
34,17.285138,0.828283,0.907970,relu,sigmoid,0.6,32,nadam,two,1,0.032236,0.016118
21,11.450269,0.827160,0.897027,relu,sigmoid,0.4,32,adagrad,two,2,0.036602,0.018301
32,14.476058,0.826038,0.892820,relu,sigmoid,0.6,32,rmsprop,two,3,0.034406,0.017203
25,13.721734,0.823793,0.909933,relu,sigmoid,0.5,32,adam,two,4,0.032762,0.016381
33,16.872469,0.823793,0.896186,relu,sigmoid,0.6,32,adadelta,two,4,0.035072,0.017536
16,9.869362,0.821549,0.898712,relu,sigmoid,0.3,32,adagrad,two,6,0.042841,0.021421
35,16.242781,0.821549,0.890013,relu,sigmoid,0.7,32,adam,two,6,0.036148,0.018074
12,9.129408,0.821549,0.905165,relu,sigmoid,0.2,32,rmsprop,two,6,0.044222,0.022111
36,16.231468,0.820426,0.875142,relu,sigmoid,0.7,32,adagrad,two,9,0.035659,0.017829
20,12.375190,0.820426,0.916107,relu,sigmoid,0.4,32,adam,two,9,0.040325,0.020162


## DNN in SciKit Learn (MLPClassifier)

In [6]:
from sklearn.neural_network import MLPClassifier

### Data Type 1

In [22]:
X=data_1.drop('Survived',axis=1).as_matrix()
Y=data_1['Survived'].as_matrix()

MLP_class = MLPClassifier(max_iter=1000)

param_grid ={
    'hidden_layer_sizes': [(100,),(64,64),(32,32),(16,16),(64,),(32,),(16,)],
    'activation':['logistic', 'tanh', 'relu'],
    'solver':['lbfgs','adam'],
    'alpha':[10**x for x in range(-4,0)],
    }

gscv=GridSearchCV(MLP_class,param_grid,scoring='accuracy',cv=cv_k_global,verbose=1,n_jobs=-1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)
    
if(dataset=='new'):
    results_df.to_csv('./CV_MLP_new_data1.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_MLP_data1.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_MLP_deck_data1.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_MLP_deck_new_data1.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 168 candidates, totalling 1848 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   46.2s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 1848 out of 1848 | elapsed:  7.4min finished


,mean_fit_time,mean_test_score,mean_train_score,param_activation,param_alpha,param_hidden_layer_sizes,param_solver,rank_test_score,std_test_score,error
115,1.131801,0.813692,0.830080,relu,0.0001,"(64, 64)",adam,1,0.041791,0.020896
52,2.992623,0.811448,0.918631,logistic,0.1,"(32,)",lbfgs,2,0.043823,0.021912
117,0.649839,0.808081,0.820763,relu,0.0001,"(32, 32)",adam,3,0.039292,0.019646
129,0.993892,0.806958,0.829404,relu,0.001,"(64, 64)",adam,4,0.039088,0.019544
131,0.749606,0.806958,0.822561,relu,0.001,"(32, 32)",adam,4,0.029104,0.014552
50,5.173392,0.805836,0.918294,logistic,0.1,"(64,)",lbfgs,6,0.039555,0.019777
157,0.938629,0.804714,0.821774,relu,0.1,"(64, 64)",adam,7,0.045651,0.022825
161,0.438005,0.803591,0.810552,relu,0.1,"(16, 16)",adam,8,0.034715,0.017358
159,0.693215,0.803591,0.818855,relu,0.1,"(32, 32)",adam,8,0.042108,0.021054
54,1.563033,0.803591,0.912235,logistic,0.1,"(16,)",lbfgs,8,0.037800,0.018900


### Data Type 2

In [23]:
X=data_2.drop('Survived',axis=1).as_matrix()
Y=data_2['Survived'].as_matrix()

MLP_class = MLPClassifier(max_iter=1000)

param_grid ={
    'hidden_layer_sizes': [(100,),(64,64),(32,32),(16,16),(64,),(32,),(16,)],
    'activation':['logistic', 'tanh', 'relu'],
    'solver':['lbfgs','adam'],
    'alpha':[10**x for x in range(-4,0)],
    }

gscv=GridSearchCV(MLP_class,param_grid,scoring='accuracy',cv=cv_k_global,verbose=1,n_jobs=-1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)
    
if(dataset=='new'):
    results_df.to_csv('./CV_MLP_new_data2.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_MLP_data2.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_MLP_deck_data2.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_MLP_deck_new_data2.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 168 candidates, totalling 1848 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   49.1s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 1848 out of 1848 | elapsed:  7.7min finished


,mean_fit_time,mean_test_score,mean_train_score,param_activation,param_alpha,param_hidden_layer_sizes,param_solver,rank_test_score,std_test_score,error
129,1.296036,0.819304,0.842425,relu,0.001,"(64, 64)",adam,1,0.043336,0.021668
157,1.162913,0.818182,0.833895,relu,0.1,"(64, 64)",adam,2,0.036392,0.018196
121,0.596091,0.810325,0.822334,relu,0.0001,"(64,)",adam,3,0.042026,0.021013
149,0.620633,0.810325,0.827048,relu,0.01,"(64,)",adam,3,0.029656,0.014828
83,0.365907,0.806958,0.810102,tanh,0.001,"(16,)",adam,5,0.037770,0.018885
133,0.374923,0.806958,0.820543,relu,0.001,"(16, 16)",adam,5,0.045012,0.022506
161,0.395023,0.805836,0.819752,relu,0.1,"(16, 16)",adam,7,0.043913,0.021957
115,1.225707,0.805836,0.840291,relu,0.0001,"(64, 64)",adam,7,0.043902,0.021951
159,0.589024,0.805836,0.819078,relu,0.1,"(32, 32)",adam,7,0.040217,0.020108
145,0.644413,0.805836,0.824467,relu,0.01,"(32, 32)",adam,7,0.038661,0.019331


### Data Type 3

In [24]:
X=data_3.drop('Survived',axis=1).as_matrix()
Y=data_3['Survived'].as_matrix()

MLP_class = MLPClassifier(max_iter=1000)

param_grid ={
    'hidden_layer_sizes': [(100,),(64,64),(32,32),(16,16),(64,),(32,),(16,)],
    'activation':['logistic', 'tanh', 'relu'],
    'solver':['lbfgs','adam'],
    'alpha':[10**x for x in range(-4,0)],
    }

gscv=GridSearchCV(MLP_class,param_grid,scoring='accuracy',cv=cv_k_global,verbose=1,n_jobs=-1)
gscv.fit(X,Y)
results_df=pd.DataFrame(gscv.cv_results_)
    
if(dataset=='new'):
    results_df.to_csv('./CV_MLP_new_data3.csv')
if(dataset=='old'):
    results_df.to_csv('./CV_MLP_data3.csv')
if(dataset=='deck'):
    results_df.to_csv('./CV_MLP_deck_data3.csv')
if(dataset=='deck_new'):
    results_df.to_csv('./CV_MLP_deck_new_data3.csv')

results_df['error']=results_df['std_test_score']/2
results_df.sort_values('rank_test_score',axis=0).drop(['mean_score_time','params','split0_test_score', 'split0_train_score', 'split10_test_score',
       'split10_train_score', 'split1_test_score', 'split1_train_score',
       'split2_test_score', 'split2_train_score', 'split3_test_score',
       'split3_train_score', 'split4_test_score', 'split4_train_score',
       'split5_test_score', 'split5_train_score', 'split6_test_score',
       'split6_train_score', 'split7_test_score', 'split7_train_score',
       'split8_test_score', 'split8_train_score', 'split9_test_score',
       'split9_train_score','std_fit_time', 'std_score_time', 'std_train_score'],axis=1).head(10)

Fitting 11 folds for each of 168 candidates, totalling 1848 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   50.1s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 1848 out of 1848 | elapsed:  8.0min finished


,mean_fit_time,mean_test_score,mean_train_score,param_activation,param_alpha,param_hidden_layer_sizes,param_solver,rank_test_score,std_test_score,error
157,1.189270,0.832772,0.841528,relu,0.1,"(64, 64)",adam,1,0.035649,0.017824
159,0.633867,0.822671,0.831759,relu,0.1,"(32, 32)",adam,2,0.021677,0.010839
163,0.398321,0.821549,0.821212,relu,0.1,"(64,)",adam,3,0.025520,0.012760
149,0.523268,0.819304,0.825590,relu,0.01,"(64,)",adam,4,0.038857,0.019428
165,0.420474,0.815937,0.821999,relu,0.1,"(32,)",adam,5,0.022936,0.011468
115,0.985096,0.815937,0.842651,relu,0.0001,"(64, 64)",adam,5,0.026212,0.013106
155,0.710187,0.815937,0.828619,relu,0.1,"(100,)",adam,5,0.023532,0.011766
151,0.514767,0.815937,0.824916,relu,0.01,"(32,)",adam,5,0.020921,0.010460
129,1.073928,0.815937,0.842085,relu,0.001,"(64, 64)",adam,5,0.038409,0.019205
145,0.636711,0.815937,0.834459,relu,0.01,"(32, 32)",adam,5,0.014843,0.007421
